In this tutorial, we will create a pipeline that filters the input data, then calculates the covariance matrix.

In [1]:
# this is to setup the path so we can import the mindpype library
import os; os.sys.path.append(os.path.dirname(os.path.abspath('.')))

In [2]:
import mindpype as mp

import numpy as np
from scipy import signal

We will create a function to manually filter the data and compute the covariance matrix. We will use this function to compare to the output of our graph to check if our pipeline produces an accurate result.

In [3]:
# Create a manual computation function to compare the output of the graph with
def manual_computation(input_data):

    # first filter the data
    sos = signal.butter(4,(8,35),btype='bandpass',output='sos',fs=250)
    filtered_data = signal.sosfilt(sos,input_data,axis=1)
    cov_mat = np.cov(filtered_data)

    return cov_mat

The first step to creating a pipeline is to create a session, which serves as a sandbox for all components in the pipeline. After creating the session we will create a graph which represents the pipeline.

In [4]:
# create a session
s = mp.Session.create()
trial_graph = mp.Graph.create(s)

Next we will create our data containers for the graph. 

Our pipeline will take input data and will output a covariance matrix so we will create a tensor object for our input and a tensor object for our output. For our input data we will randomly generate an ndarray with dimensions 12 and 500 and create a tensor from this ndarray using the ```create_from_data()``` factory method. For our output data we will create a 12 x 12 tensor using the ```create()``` factory method.

We will have one intermediate object that contains the filtered data that will be passed to our covariance kernel. Since we only need this intermediate object for the next calculation and will not need to access it at a later time, we will create a virtual tensor to store it using the ```create_virtual()``` factory method.

In [5]:
# Create dummy input data and tensor
input_data = np.random.randn(12,500)
input_tensor = mp.Tensor.create_from_data(s,input_data)

# Create output tensor
output_tensor = mp.Tensor.create(s,(12,12))

# Create virtual tensor for intermediate output
intermediate_tensor = mp.Tensor.create_virtual(s)

Next we will create a filter. We first set the parameter values for the filter, then we will use these values to create a butterworth filter using the ```create_butter()``` factory method.

In [6]:
# create a filter
order = 4
bandpass = (8,35) # in Hz
fs = 250
filter_obj = mp.Filter.create_butter(s,order,bandpass,btype='bandpass',fs=fs,implementation='sos')

Now we will add the associated kernels to our graph using the ```add_to_graph()``` factory methods. Each kernel that we add to the graph represents a node that will execute a process. 

For our pipeline, we will first filter the data and store it in our intermediate tensor. Then we will compute the covariance matrix using the filtered values in the intermediate tensor. 

In [7]:
# add the nodes
mp.kernels.FilterKernel.add_to_graph(trial_graph,input_tensor,filter_obj,intermediate_tensor)
mp.kernels.CovarianceKernel.add_to_graph(trial_graph,intermediate_tensor,output_tensor)

With all of our nodes added to our graph, we will then verify the graph using the ```verify()``` method. Verifying the graph orders the nodes for execution and ensure that the inputs and outputs of each processing node are appropriately typed and sized.

In [8]:
# verify the session (i.e. schedule the nodes)
trial_graph.verify()

Verifying kernel Filter...
Verifying kernel Covariance...


For our pipeline, no nodes require initialization or training so initializing the graph is not required. However, if our graph had nodes that needed initialization/training such as a classifier node, then we would need to initialize the graph like so: 

In [9]:
# initializing the graph - not required since there are no nodes that require initialization/training data
trial_graph.initialize()

We are now ready to run our pipeline. To run the graph for the provided input data, we use the ```execute()``` method.

In [10]:
# RUN!
trial_graph.execute()

Executing trial with label: None


Using the manual computation function that we defined earlier, we can compare the ground truth covariance matrix (from the manual calculation) to the covarinace matrix produced by our pipeline to ensure the difference is negligible.

In [11]:
# compare the output with manual calculation
ground_truth = manual_computation(input_data)

max_diff = np.max(np.abs(output_tensor.data - ground_truth))
print(max_diff)

# Check if the difference is within the machine epsilon
if max_diff <= np.finfo(np.float64).eps:
    print("Test Passed =D")
else:
    print("Test Failed D=")


0.0
Test Passed =D
